# Equation Solver And Tkinter GUI

In [ ]:
import os
import numpy as np
import cv2
import re
import cmath
from PIL import ImageGrab, Image
import tkinter as tk
from tkinter import Button, Label, Canvas, W
from tensorflow import keras
from tensorflow.keras.models import model_from_json, load_model
from tensorflow.keras.utils import to_categorical
from sympy import sympify, Eq, solve

# -----------------------
# 1) Load model safely
# -----------------------
MODEL_DIR = "model"
FULL_MODEL_PATH = os.path.join(MODEL_DIR, "full_model.h5")
JSON_PATH = os.path.join(MODEL_DIR, "model.json")
WEIGHTS_PATH = os.path.join(MODEL_DIR, "model_weights.weights.h5")  # required extension

model = None
if os.path.exists(FULL_MODEL_PATH):
    print("Loading full model:", FULL_MODEL_PATH)
    model = load_model(FULL_MODEL_PATH)
elif os.path.exists(JSON_PATH) and os.path.exists(WEIGHTS_PATH):
    print("Loading model from JSON + weights")
    with open(JSON_PATH, 'r') as f:
        loaded_json = f.read()
    model = model_from_json(loaded_json)
    model.load_weights(WEIGHTS_PATH)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
else:
    raise FileNotFoundError(
        "No saved model found. Place model/full_model.h5 or (model/model.json + model/model_weights.weights.h5)."
    )

# Labels mapping (keep same as training)
labels = ['0','1','2','3','4','5','6','7','8','9','+','-','x']

# -----------------------
# 2) Utility: predict single image
# -----------------------
def predict_image(img_28x28_gray):
    """
    img_28x28_gray: numpy array shape (28,28), dtype uint8 or float
    returns: predicted label string
    """
    # normalize
    x = img_28x28_gray.astype("float32") / 255.0
    
    # flatten to match model input
    x = x.reshape(1, 784)   # instead of (1,28,28,1)
    
    probs = model.predict(x)
    idx = int(np.argmax(probs, axis=1)[0])
    return labels[idx]


# -----------------------
# 3) Equation solver (your existing class, slightly cleaned)
# -----------------------
class Solver:
    def __init__(self, equation):
        self.equation = str(equation)
        self.leftEqu = []

    def convertEquationIntoGeneralForm(self):
        leftSide, rightSide = '', ''
        equalIndx = self.equation.index('=')
        leftSide = self.equation[0:equalIndx]
        rightSide = self.equation[equalIndx+1:len(self.equation)]

        if rightSide and (rightSide[0].isalpha() or rightSide[0].isdigit()):
            rightSide = '+' + rightSide

        # move everything to left
        for i in range(0, len(rightSide)):
            if rightSide[i] == '+':
                rightSide = rightSide[0:i] + '-' + rightSide[i+1:len(rightSide)]
            elif rightSide[i] == '-':
                rightSide = rightSide[0:i] + '+' + rightSide[i+1:len(rightSide)]
            leftSide += rightSide[i]

        self.equation = leftSide + '=' + '0'
        self.leftEqu = leftSide

    def solveEquation(self):
        self.convertEquationIntoGeneralForm()
        sympy_eq = sympify("Eq(" + self.equation.replace("=", ",") + ")")
        roots = solve(sympy_eq)
        return roots

# -----------------------
# 4) TK GUI + drawing handlers
# -----------------------
root = tk.Tk()
root.resizable(0,0)
root.title('Equation Solver')

lasx, lasy = None, None

cv = Canvas(root, width=1200, height=500, bg='white')
cv.grid(row=0, column=0, pady=2, sticky=W, columnspan=2)

cve = Label(root, font=("Helvetica", 16))
cve2 = Label(root)
cve.grid(row=0, column=1, pady=1, padx=1)
cve2.grid(row=1, column=1, pady=1, padx=1)

def activate_event(event):
    global lasx, lasy
    lasx, lasy = event.x, event.y

def draw_smth(event):
    global lasx, lasy
    # draw line on canvas
    cv.create_line((lasx, lasy, event.x, event.y), fill='black', width=8)
    lasx, lasy = event.x, event.y

cv.bind('<Button-1>', activate_event)
cv.bind('<B1-Motion>', draw_smth)

# -----------------------
# 5) Save canvas to file
# -----------------------
def save_canvas(filename="canvas.jpg"):
    widget = cv
    # coordinates of canvas on screen
    x = root.winfo_rootx() + widget.winfo_x()
    y = root.winfo_rooty() + widget.winfo_y()
    x1 = x + widget.winfo_width()
    y1 = y + widget.winfo_height()
    # Grab and save
    ImageGrab.grab(bbox=(x, y, x1, y1)).save(filename)
    print(f"[INFO] Saved {filename}")

# -----------------------
# 6) Main prediction & solve routine
# -----------------------
def solution():
    # 1) save the canvas
    save_canvas("canvas.jpg")

    # 2) read & preprocess for contours
    img = cv2.imread("canvas.jpg", cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("Failed to read canvas.jpg")
        return

    # invert so strokes are white on black (depends on how you trained)
    img = cv2.bitwise_not(img)

    # threshold
    _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # find contours - external only
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print("No contours found.")
        return

    # sort left-to-right
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

    # extract each component, resize to 28x28 and predict
    mainEquation = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        pad = 8  # give some padding
        x1 = max(x - pad, 0)
        y1 = max(y - pad, 0)
        x2 = min(x + w + pad, thresh.shape[1])
        y2 = min(y + h + pad, thresh.shape[0])

        roi = thresh[y1:y2, x1:x2]
        if roi.size == 0:
            continue

        # Resize preserving aspect ratio: center it in 28x28
        roi_resized = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)

        # Optionally: apply dilation/erode or smoothing if needed
        # Normalize if model was trained with 0-1 inputs
        label = predict_image(roi_resized)
        mainEquation.append(label)

    # Build equation string from predicted tokens (your original logic is complex; here's a simple join)
    # The original logic attempted to infer multiplication/exponent; below is a simpler reconstruction.
    eq_str = "".join(mainEquation)
    print("Predicted tokens:", mainEquation)
    print("Raw equation string:", eq_str)

    # Try to clean double '-' representing '=' etc. (adapt as needed)
    # Replace any accidental sequences '=-' etc. if necessary
    # For now assume user wrote something like "x+2=5"
    try:
        solver = Solver(eq_str)
        roots = solver.solveEquation()
        # format roots
        roots_str = ", ".join([str(r) for r in roots]) if roots else "No solution / complex"
    except Exception as e:
        roots_str = f"Error solving: {e}"
        print("Solver error:", e)

    # show results
    cve2.configure(text='Your Equation is : ' + eq_str)
    cve.configure(text='Result : ' + roots_str + '\n')

# Buttons
btn_save = Button(text="Save", command=lambda: save_canvas("canvas.jpg"), bg='#6495ED', fg='white')
btn_save.grid(row=2, column=0, pady=1, padx=1)

btn_predict = Button(text="Predict", command=solution, bg='#6495ED', fg='white')
btn_predict.grid(row=2, column=1, pady=1, padx=1)

root.mainloop()


Loading model from JSON + weights
[INFO] Saved canvas.jpg
[INFO] Saved canvas.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted tokens: ['9', '2', '0', '0', '2', '0', '7', '2', '0', '2', '0', '7', '7', '2']
Raw equation string: 92002072020772
Solver error: substring not found


In [ ]:
labels = [
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9",
    "+", "-", "*", "/", "^", "="
]


In [ ]:
from PIL import Image,ImageDraw,ImageGrab

In [ ]:
def activate_event(event):
        global lasx,lasy
        lasx,lasy=event.x,event.y   

In [ ]:
def draw_smth(event):
    global lasx,lasy
    cv.create_line((lasx,lasy,event.x,event.y),fill='black',width=4)
    #cv.draw.line([x1, y1, x2, y2], fill="black", width=7)
    lasx,lasy=event.x,event.y

In [ ]:
def save():
    filename="canvas.jpg"
    widget=cv
    x=root.winfo_rootx()+widget.winfo_x()+50
    y=root.winfo_rooty()+widget.winfo_y()+50
    x1=x+widget.winfo_width()
    y1=y+widget.winfo_height()
    
    ImageGrab.grab().crop((x,y,x1,y1)).save(filename)

In [ ]:
from sympy import *

class Solver:

    def __init__(self, equation):
        self.equation = str(equation)
        self.leftEqu = []

    def convertEquationIntoGeneralForm(self):

        leftSide, rightSide = '', ''
        equalIndx = self.equation.index('=')
        leftSide = self.equation[0:equalIndx]
        rightSide = self.equation[equalIndx+1:len(self.equation)]

        if rightSide[0].isalpha() or rightSide[0].isdigit():
            rightSide = '+' + rightSide

        for i in range(0, len(rightSide)):
            if rightSide[i] == '+':
                rightSide = rightSide[0:i] + '-' + rightSide[i+1:len(rightSide)]
            elif rightSide[i] == '-':
                rightSide = rightSide[0:i] + '+' + rightSide[i+1:len(rightSide)]
            leftSide += rightSide[i]

        self.equation = leftSide + '=' + '0'
        self.leftEqu = leftSide

    def solveEquation(self):

        self.convertEquationIntoGeneralForm()
        sympy_eq = sympify("Eq(" + self.equation.replace("=", ",") + ")")
        roots = solve(sympy_eq)
        
        return roots      

In [ ]:
def predictFromArray(arr):
    """
    arr can be (28,28), (1,28,28), or (1,28,28,1)
    This will flatten to (1,784) for the model
    """
    import numpy as np

    if arr.ndim == 4:        # (1,28,28,1)
        arr = arr.reshape(1, 784)
    elif arr.ndim == 3:      # (1,28,28)
        arr = arr.reshape(1, 784)
    elif arr.ndim == 2:      # (28,28)
        arr = arr.reshape(1, 784)

    arr = arr.astype("float32") / 255.0  # normalize
    probs = model.predict(arr, verbose=0)
    return int(np.argmax(probs, axis=-1)[0])


In [ ]:

import re
from sympy import symbols, Eq, solve, sympify

def clean_equation(equation: str) -> str:
    """
    Cleans up OCR text to make it look like a proper math equation.
    """
    # Normalize letters
    equation = equation.replace("X", "x")  # Capital X → lowercase x
    equation = equation.replace("^", "**") # Caret → exponent

    # Remove unwanted characters (OCR noise)
    equation = re.sub(r"[^0-9xX\+\-\*/=\^\(\)]", "", equation)

    return equation

def solve_equation(equation: str):
    try:
        # Step 1: Clean OCR output
        equation = clean_equation(equation)

        # Step 2: Validate equation
        if "=" not in equation:
            return f"❌ OCR Error: '=' sign missing in detected text ({equation})"
        if "x" not in equation.lower():
            return f"❌ OCR Error: variable 'x' missing in detected text ({equation})"

        # Step 3: Split into LHS and RHS
        lhs, rhs = equation.split("=")

        # Step 4: Convert into Sympy equation
        x = symbols('x')
        expr = Eq(sympify(lhs), sympify(rhs))

        # Step 5: Solve
        solutions = solve(expr, x)
        return solutions

    except Exception as e:
        return f"❌ Error solving equation: {str(e)}"



# ================== GUI Update ==================
def solving(equ):
    result = solve_equation(equ)
    cve2.configure(text=f"Your Equation is : {equ}")
    cve.configure(text=f"Result : {result}\n")


# ================== Image → Equation ==================
def solution():
    img = cv2.imread('canvas.jpg', cv2.IMREAD_GRAYSCALE)
    img = ~img
    ret, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    ctrs, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])

    img_data = []
    rects = []
    for c in cnt:
        x, y, w, h = cv2.boundingRect(c)
        rects.append([x, y, w, h])

    for r in rects:
        x, y, w, h = r
        roi = thresh[y:y + h + 10, x:x + w + 10]
        roi = cv2.resize(roi, (28, 28))
        roi = np.reshape(roi, (1, 28, 28))
        img_data.append(roi)

    # Predict each symbol
    mainEquation = []
    for data in img_data:
        idx = predictFromArray(data)  # <- your model prediction
        mainEquation.append(labels[idx])

    # ---- Build equation string ----
    StringEquation = ""
    for i in range(len(mainEquation)):
        a = mainEquation[i]
        if (not a.isdigit() and not a.isalpha() and i < len(mainEquation) - 1):
            if (a == mainEquation[i + 1] == '-'):
                StringEquation += '='
            else:
                StringEquation += a
        elif a.isalpha():
            if (i > 0 and mainEquation[i - 1].isdigit()):
                StringEquation += "*" + a
            else:
                StringEquation += a
        elif a.isdigit():
            if (i > 0 and mainEquation[i - 1].isdigit()):
                StringEquation += a
            elif (i > 0 and mainEquation[i - 1].isalpha()):
                StringEquation += "^" + a
            else:
                StringEquation += a

    # ---- Clean up equation (remove duplicate "=") ----
    if StringEquation.count("=") > 1:
        # keep only first "="
        left, right = StringEquation.split("=", 1)
        StringEquation = left + "=" + right.replace("=", "")

    equ = StringEquation

    # ✅ Pass to solver
    solving(equ)


In [ ]:
from tkinter import*

root = Tk()
root.resizable(0, 0)
root.title('Equation Solver')

lasx, lasy = None, None

cv = Canvas(root, width=1200, height=500, bg='white')
cv.grid(row=0, column=0, pady=2, sticky=W, columnspan=2)

cve2 = Label(root)
cve = Label(root, font=("Helvetica", 16))
cve.grid(row=0, column=1, pady=1, padx=1)
cve2.grid(row=1, column=1, pady=1, padx=1)

cv.bind('<Button-1>', activate_event)
cv.bind('<B1-Motion>', draw_smth)

btn_save = Button(text="Save", command=save, bg='#6495ED', fg='White')
btn_save.grid(row=2, column=0, pady=1, padx=1)

btn_predict = Button(text="Predict", command=solution, bg='#6495ED', fg='White')
btn_predict.grid(row=2, column=1, pady=1, padx=1)

root.mainloop()
